In [3]:
%pip install tensorflow keras pandas numpy scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [7]:
# Đọc dữ liệu MovieLens 1M
file_path = r"C:\Thesis\Sample Project\ratings.dat"
column_names = ["user_id", "item_id", "rating", "timestamp"]
data = pd.read_csv(file_path, sep='::', names=column_names, engine='python')

data = data.drop(columns=["timestamp"])  # Bỏ cột timestamp

In [10]:

# Load thêm dữ liệu người dùng và phim
users = pd.read_csv(r"C:\Thesis\Sample Project\users.dat", sep='::', engine='python',
                    names=["user_id", "gender", "age", "occupation", "zip"], encoding='latin-1')
movies = pd.read_csv(r"C:\Thesis\Sample Project\movies.dat", sep='::', engine='python',
                     names=["item_id", "title", "genres"], encoding='latin-1') 

# Encode các cột phân loại
from sklearn.preprocessing import LabelEncoder
gender_enc = LabelEncoder()
occupation_enc = LabelEncoder()
age_enc = LabelEncoder()
genre_enc = LabelEncoder()

users["gender"] = gender_enc.fit_transform(users["gender"])
users["occupation"] = occupation_enc.fit_transform(users["occupation"])
users["age"] = age_enc.fit_transform(users["age"])

# Với genres có nhiều thể loại, tạm thời chỉ lấy thể loại đầu tiên
movies["genres"] = movies["genres"].apply(lambda x: x.split('|')[0])
movies["genres"] = genre_enc.fit_transform(movies["genres"])

# Gộp dữ liệu lại
data = data.merge(users, on="user_id")
data = data.merge(movies, on="item_id")


In [11]:

# Mã hóa ID người dùng và phim
user_encoder = LabelEncoder()
data["user_id"] = user_encoder.fit_transform(data["user_id"])

item_encoder = LabelEncoder()
data["item_id"] = item_encoder.fit_transform(data["item_id"])

num_users = data["user_id"].nunique()
num_items = data["item_id"].nunique()
# Chia tập huấn luyện và kiểm tra
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [13]:
# Xây dựng mô hình DeepFM (chỉ giữ phần DeepFM)
def build_deepfm(num_users, num_items, embedding_dim=8, hidden_dims=[32, 16], dropout=0.5):
    # Input layers
    user_input = layers.Input(shape=(1,), name='user_input')
    item_input = layers.Input(shape=(1,), name='item_input')

    # Embedding layers
    user_emb = layers.Embedding(num_users, embedding_dim)(user_input)
    item_emb = layers.Embedding(num_items, embedding_dim)(item_input)

    # Flatten embeddings
    user_emb_flat = layers.Flatten()(user_emb)
    item_emb_flat = layers.Flatten()(item_emb)

    # ⭐ Thêm phần FM (Factorization Machine) ⭐
    interaction = layers.Dot(axes=1)([user_emb_flat, item_emb_flat])# Nhân chéo embedding

    # Deep component (DNN)
    concat_features = layers.Concatenate()([user_emb_flat, item_emb_flat])
    dnn = concat_features
    for dim in hidden_dims:  # 🔧 Sửa đổi: DNN tổng quát hơn với danh sách hidden_dims
        dnn = layers.Dense(dim, activation='relu')(dnn)
        dnn = layers.Dropout(dropout)(dnn)
    dnn = layers.Dense(1)(dnn)

    # Kết hợp các thành phần
    output = layers.Add()([interaction, dnn])
    output = layers.Activation('sigmoid')(output)

    # Xây dựng mô hình
    model = keras.Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC()])

    return model

In [14]:
# 🔧 Sửa đổi: Cho phép tùy chỉnh embedding_dim, hidden_dims, và dropout
embedding_dim = 8
hidden_dims = [64,32]
dropout = 0.5

# Khởi tạo mô hình DeepFM
model = build_deepfm(num_users, num_items, embedding_dim, hidden_dims, dropout)

# Chuyển đổi dữ liệu đầu vào thành định dạng phù hợp
X_train = [train["user_id"].values, train["item_id"].values]
y_train = (train["rating"].values >= 4).astype(int)  # Chuyển rating thành nhãn nhị phân

X_test = [test["user_id"].values, test["item_id"].values]
y_test = (test["rating"].values >= 4).astype(int)  # Chuyển rating thành nhãn nhị phân
# Huấn luyện mô hình
history = model.fit(X_train, y_train, batch_size=256, epochs=10, validation_data=(X_test, y_test), verbose=1)  # ⭐ Giữ nguyên cách gọi của file gốc ⭐

# Đánh giá mô hình
loss, accuracy, auc = model.evaluate(X_test, y_test)  # ⭐ Lấy thêm giá trị AUC ⭐
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")  # ⭐ In ra AUC ⭐



Epoch 1/10


3126/3126 [==============================] - 11s 3ms/step - loss: 0.5671 - accuracy: 0.7092 - auc: 0.7668 - val_loss: 0.5422 - val_accuracy: 0.7242 - val_auc: 0.7888
Epoch 2/10
3126/3126 [==============================] - 9s 3ms/step - loss: 0.5414 - accuracy: 0.7284 - auc: 0.7923 - val_loss: 0.5375 - val_accuracy: 0.7269 - val_auc: 0.7929
Epoch 3/10
3126/3126 [==============================] - 9s 3ms/step - loss: 0.5355 - accuracy: 0.7319 - auc: 0.7978 - val_loss: 0.5355 - val_accuracy: 0.7273 - val_auc: 0.7953
Epoch 4/10
3126/3126 [==============================] - 9s 3ms/step - loss: 0.5304 - accuracy: 0.7353 - auc: 0.8023 - val_loss: 0.5333 - val_accuracy: 0.7295 - val_auc: 0.7973
Epoch 5/10
3126/3126 [==============================] - 9s 3ms/step - loss: 0.5253 - accuracy: 0.7390 - auc: 0.8070 - val_loss: 0.5313 - val_accuracy: 0.7312 - val_auc: 0.8001
Epoch 6/10
3126/3126 [==============================] - 9s 3ms/step - loss: 0.5201 - accuracy: 0.7432 - auc: 0.8118

In [15]:
# Lưu mô hình
model.save("deepfm_model.keras")